#### Zadanie:
Identifikujte sedenia na základe 10 minútového odhadu dĺžky času sedenia (SLength).

Odovzdajte zdrojový kód programu + logovací súbor doplnený o identifikáciu používateľov (v jednom zip).

Príkazy v Pythone vhodné pre riešenie tohto zadania:

`import pandas as pd
pd.read_csv() - načítanie log súboru`

In [1]:
import pandas as pd
import numpy as np

In [2]:
# opening the file we will be working with and assigning it to the variabel file for latter use
file = pd.read_csv("11_Identifikácia_sedení_STT_Q.csv",  encoding="utf-8",  escapechar=";", index_col=False)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# session estimate of 10 mins in seconds (10*60)
session_estimate = 600

In [4]:
# initializing the empty array for SLength id numbers
# initializing the temp var SLength_id to store temp ID numbers of the SLenght position index
SLength = []
SLength_ID = 0

# temp variables for comparison
previous_seq_id = 0
previous_length = 0
cumulative_length = 0
# we increment when seq_id != previous_seq_id

# we increment when seq_id == previous_seq_id and:
#     1. previous_length is NaN and length variable is bigger than session_estimate value
#     2. previous_length is NaN and length is NaN
#     3. previous_length is more than session_estimate 
#     4. previous_length is less than session_estimate and length is NaN
#     5. cumulative length time is more than session_estimate

for seq_id, length in zip(file["SEQUENCE_ID"], file["Length"]):
    if ((previous_seq_id != seq_id)
    or ((previous_seq_id == seq_id) and (np.isnan(previous_length) and length > session_estimate))
    or ((previous_seq_id == seq_id) and (np.isnan(previous_length) and np.isnan(length)))
    or ((previous_seq_id == seq_id) and (previous_length > session_estimate))
    or ((previous_seq_id == seq_id) and (np.isnan(previous_length) and length < session_estimate))
    or ((previous_seq_id == seq_id) and (cumulative_length > session_estimate))):
    
        SLength_ID += 1
        SLength.append(SLength_ID)
        previous_seq_id = seq_id
        previous_length = length
        cumulative_length = 0

    else:
        SLength.append(SLength_ID)
        previous_seq_id = seq_id
        previous_length = length
        cumulative_length += length

In [5]:
# Inserting new column into the dataset
file.insert(11, 'SLength', SLength)

In [6]:
# writing the file to new csv, escape char ";" to not get User-Agent divided on export/write
file.to_csv("12_Identifikácia_sedení_SLength.csv", encoding="utf-8",  escapechar=";", index=False)